In [1]:
#default_exp oo_hp

In [2]:
# handle already sampled HP
# go over classes and find invalid assumptions

In [3]:
# handle names for expressions - change expression names within expression

In [4]:
#export
import fastai2
from fastai2.tabular.all import *
from fastai2.metrics import *

In [5]:
#export
from copy import deepcopy

In [6]:
#export
import optuna

In [7]:
#export
SEED = 42

In [8]:
from nbdev import export

In [9]:
from nbdev.export import *

In [10]:
#export
from collections.abc import Iterable

# HyPSTER Classes

In [11]:
#export
from inspect import signature, Parameter
import functools

def auto_assign(func):
    # Signature:
    sig = signature(func)
    for name, param in sig.parameters.items():
        if param.kind in (Parameter.VAR_POSITIONAL, Parameter.VAR_KEYWORD):
            raise RuntimeError('Unable to auto assign if *args or **kwargs in signature.')
    # Wrapper:
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):
        for i, (name, param) in enumerate(sig.parameters.items()):
            # Skip 'self' param:
            if i == 0: continue
            # Search value in args, kwargs or defaults:
            if i - 1 < len(args):
                val = args[i - 1]
            elif name in kwargs:
                val = kwargs[name]
            else:
                val = param.default
            setattr(self, name, val)
        func(self, *args, **kwargs)
    return wrapper

In [12]:
#export
class HypsterBase(object):
    def __init__(self): return
#TODO: add stuff here

In [13]:
#export
class HpExpression(object):
    def __init__(self, exp1, exp2):
        self.exp1 = exp1; self.exp2 = exp2    
    
    def sample(self, trial): raise NotImplementedError
    
    def get_name(self):
        if self.name is not None: return self.name

        name = ""
        if self.exp1 is not None and isinstance(self.exp1, HpExpression) and hasattr(self.exp1, "name"):
            name += self.exp1.name
        if self.exp2 is not None and isinstance(self.exp2, HpExpression) and hasattr(self.exp2, "name"):
            if len(name) > 0:
                name +=  "_"
            name += self.exp2.name
        self.name = name
        return self.name
        #TODO: refactor
        
    def __add__(self, other):  return AddExpression(self, other)
    def __radd__(self, other): return AddExpression(other, self)
    def __sub__(self, other):  return SubExpression(self, other)
    def __rsub__(self, other): return SubExpression(other, self)
    def __mul__(self, other):  return MulExpression(self, other)
    def __rmul__(self, other): return MulExpression(other, self)
    def __div__(self, other):  return DivExpression(self, other)
    def __rdiv__(self, other): return DivExpression(other, self)
    def __pow__(self, other):  return PowExpression(self, other)
    def __rpow__(self, other): return PowExpression(other, self)

In [14]:
#export
HYPSTER_TYPES = (HypsterBase, HpExpression)

In [15]:
#export
class SubExpression(HpExpression):
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 - exp2

In [16]:
#export
class AddExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 + exp2

In [17]:
#export
class MulExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 * exp2

In [18]:
#export
class DivExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 / exp2

In [19]:
#export
class PowExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 ** exp2

In [20]:
#TODO: add round, int, floor etc...
#TODO: add Brackets () etc...?

In [21]:
#export
class HpFloat(HpExpression):
    @auto_assign
    def __init__(self, name, low, high, log=False, step=None):
        self.result = None
        #TODO: check what's up with log and step
        #TODO: move result to HpExpression?
        
    def sample(self, trial): 
        #self.result = ifnone(self.result, trial.suggest_float(self.name, self.low, self.high))
        self.result = trial.suggest_float(self.name, self.low, self.high)
        return self.result
    
    #TODO: warn if log=True & step is not None
    #TODO: check what is the "*" in the function definition

In [22]:
#export
class HpInt(HpExpression):
    @auto_assign
    def __init__(self, name, low, high, step=1):
        self.result = None
    
    def sample(self, trial):
        #self.result = ifnone(self.result, trial.suggest_int(self.name, self.low, self.high, self.step))
        self.result = trial.suggest_int(self.name, self.low, self.high, self.step)
        return self.result

In [23]:
#export
class HpCategorical(HpExpression):
    @auto_assign
    def __init__(self, name, choices): 
        self.result = None
    
    def sample(self, trial): 
        #print(f"result = {self.result}")
        #if self.result is not None:
        #    return self.result
        
        choices           = self.choices
        name              = self.name
        optuna_valid_cats = [str, int, float, bool] #TODO: add more + move to global area
        
        if any([type(choice) not in optuna_valid_cats for choice in self.choices]):
            #TODO: add check for "choice.__name__"
            self.items_str = [choice.__name__ for choice in choices]
            self.str_dict  = dict(zip(self.items_str, choices))
            chosen_hp      = trial.suggest_categorical(name, self.str_dict)
            self.result    = self.str_dict[chosen_hp]
        else:
            self.result = trial.suggest_categorical(name, choices)
        return self.result

In [24]:
#export
class HpVarLenList(HpExpression):
    #TODO: think of a better name?
    #TODO: maje VarLenTuple
    @auto_assign
    def __init__(self, name, min_len, max_len, hp, same_value=False): pass        
    
    def sample(self, trial):
        lst_len = trial.suggest_int(self.name, self.min_len, self.max_len)
        lst = []
        if (self.same_value) or (not isinstance(self.hp, HpExpression)):
            lst = [sample_hp(self.hp, trial)] * lst_len
        else:
            for i in range(lst_len):
                hp = deepcopy(self.hp)
                hp.result = None
                hp.name = f"{hp.get_name()}_{i+1}"
                result = sample_hp(hp, trial)
                lst.append(result)
                #TODO keep self.result?
        return lst

In [25]:
#export
def list_to_tuple(lst): return (*lst, )

In [26]:
#export
class HpDict(HpExpression):
    @auto_assign
    def __init__(self, name, dct): pass
    
    def sample(self, trial):
        return {key : sample_hp(value, trial) for key, value in self.dct.items()}

In [27]:
#export
class HpTuple(HpExpression):
    @auto_assign
    def __init__(self, name, tup): pass
    
    def sample(self, trial):
        return list_to_tuple([sample_hp(item, trial) for item in self.tup])

In [28]:
#export
class HpList(HpExpression):
    @auto_assign
    def __init__(self, name, lst): pass
    
    def sample(self, trial):
        return [sample_hp(item, trial) for item in self.lst]

In [29]:
#export
class HpBool(HpCategorical):
    def __init__(self, name):
        super().__init__(name, choices=[False, True])

In [30]:
#export
class HpToggle(HpBool):
    @auto_assign
    def __init__(self, hp): return
    def sample(self, trial): return trial.suggest_categorical(f"toggle_{self.hp.name}", [False, True])
    #TODO: fix hp.name
    #TODO: support toggle in iterable, dict & upon assignment

# Tests

In [31]:
#export
DATA_STRUCTURES = (set, list, tuple, dict)

In [32]:
#export
def contains_hypster(x, types):
    if not isinstance(x, DATA_STRUCTURES):
        x = [x]
    elif isinstance(x, dict):
        x = x.values()
        
    for item in x:
        if isinstance(item, DATA_STRUCTURES):
            if contains_hypster(item, types):
                return True
        else:
            if isinstance(item, types):
                return True
    return False

In [33]:
#TODO!: check if class attributes / methods? have hypster in them

# Test contains_hypster

In [34]:
hps = []

In [35]:
hps.append(HpInt("hi!", 2, 10))

In [36]:
hps.append({"hi" : 4, "hello": HpFloat("sdf", 1.0, 2.0)})

In [37]:
hps.append(["a", 2, HpInt("hola", 1, 19)])

In [38]:
hps.append(["a", 2, {"ho": "hello"}, {"hi" : HpCategorical("d", [1,2,3])}])

In [39]:
[contains_hypster(hp, HpExpression) for hp in hps]

[True, True, True, True]

In [40]:
#export
def sample_hp(hp, trial):
    if not contains_hypster(hp, HYPSTER_TYPES):
        return hp
    #TODO: change to dynamic dispatch
    if isinstance(hp, list):
        hp = HpList("temp_iter", hp)
    elif isinstance(hp, tuple):
        hp = HpTuple("temp_tuple", hp)
    elif isinstance(hp, dict):
        hp = HpDict("temp_dict", hp)
    return hp.sample(trial)
    #TODO: handle names
    #TODO: handle list of lists
#TODO!: check if class attributes / methods? have hypster in them

In [41]:
#export
def run_hp_test(hp):
    def objective(trial):
        print(sample_hp(hp, trial))
        return 1.0        
    
    optuna.logging.set_verbosity(0)
    pruner = optuna.pruners.NopPruner()
    study = optuna.create_study(direction="maximize", pruner=pruner)
    study.optimize(objective, n_trials=5, timeout=600)

### Int

In [42]:
x = HpInt("start_mom", 2, 10)
y = HpInt("layer_size", 50, 300, 50)
z = HpVarLenList("n_layers", 1, 5, x, same_value=False)

In [43]:
run_hp_test(x)

6
3
5
3
6


In [44]:
run_hp_test(y)

50
150
250
200
50


In [45]:
run_hp_test(z)

[7, 6, 5, 10, 5]
[6, 5, 9]
[6, 2]
[6, 2, 3, 6]
[3, 9]


### Float

In [46]:
x = HpFloat("start_mom", 1.0, 2.0)
y = HpVarLenList("n_layers", 1, 5, x, same_value=False)

In [47]:
run_hp_test(x)

1.3879644664275994
1.7531483267995414
1.508005910667182
1.5193034454495609
1.7671828683108106


In [48]:
run_hp_test(y)

[1.1775870548634737, 1.8615214149551362]
[1.284111870563473]
[1.8123168700260157]
[1.1609262895136847, 1.104439518685078, 1.0655742933874026, 1.3400925333986293, 1.8611804332608208]
[1.553481026655061, 1.1174550383813238]


### Categorical

In [49]:
x = HpCategorical("cats!", ["cat", "meow", "rrrrrr"])
y = HpCategorical("cats!", [Adam, SGD, QHAdam])
z = HpVarLenList("n_layers", 1, 5, x, same_value=False)

In [50]:
run_hp_test(x)

rrrrrr
cat
meow
rrrrrr
rrrrrr


In [51]:
run_hp_test(y)

<function QHAdam at 0x00000224DBA63730>
<function Adam at 0x00000224DBA63510>
<function QHAdam at 0x00000224DBA63730>
<function Adam at 0x00000224DBA63510>
<function QHAdam at 0x00000224DBA63730>


In [52]:
run_hp_test(z)

['rrrrrr', 'cat', 'meow', 'rrrrrr']
['rrrrrr', 'cat', 'meow', 'rrrrrr', 'meow']
['meow']
['meow', 'rrrrrr', 'cat']
['rrrrrr']


### Boolean

In [53]:
x = HpBool("booli!")
y = HpVarLenList("n_layers", 1, 5, x, same_value=False)

In [54]:
run_hp_test(x)

True
False
False
True
True


In [55]:
run_hp_test(y)

[False, True]
[True, False, False, False]
[True, True, False, True]
[False, False, True]
[False, False, True, False, False]


### HpIterable

In [56]:
x = HpBool("boolean")

In [57]:
lst = [x, 5]
tup = [x, 5]
dct = {"first" : x, "second" : 5}

In [58]:
start_mom = HpFloat("start_mom", 0.85, 0.99)

In [59]:
mom = (start_mom, start_mom - 0.1)

In [60]:
run_hp_test(mom)

(0.9609741682244752, 0.8609741682244753)
(0.9664483532601778, 0.8664483532601778)
(0.9245250620164737, 0.8245250620164737)
(0.9243422688729885, 0.8243422688729886)
(0.933816574098935, 0.8338165740989351)


In [61]:
run_hp_test(lst)

[True, 5]
[False, 5]
[False, 5]
[False, 5]
[True, 5]


In [62]:
run_hp_test(tup)

[False, 5]
[True, 5]
[False, 5]
[False, 5]
[True, 5]


In [63]:
run_hp_test(dct)

{'first': True, 'second': 5}
{'first': True, 'second': 5}
{'first': False, 'second': 5}
{'first': False, 'second': 5}
{'first': True, 'second': 5}


In [64]:
#TODO consider adding "name" into HpExpression

In [65]:
#HpFunc

In [66]:
#TODO: support expressions like HpInt(...) * HpFloat(...) ?

In [67]:
notebook2script()

Converted 00_core.ipynb.
Converted 01_api.ipynb.
Converted 02_oo_hp.ipynb.
Converted 03_hypster_prepare.ipynb.
Converted 04_fastai_adult_tutorial.ipynb.
Converted 04_tabular_api.ipynb.
Converted index.ipynb.
